In [ ]:
%load_ext autoreload
%autoreload 2

import pandas as pd
from sklearn import preprocessing
import numpy as np
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import os.path
import joblib
import seaborn as sns
import matplotlib.pyplot as plt
from os.path import join
import dvu
dvu.set_style()
from collections import defaultdict
import imodelsx.process_results
results_dir = '../results/classification'

r = imodelsx.process_results.get_results_df(results_dir)
experiment_filename = '../experiments/02_classification.py'
r = imodelsx.process_results.fill_missing_args_with_default(
    r, experiment_filename)

In [ ]:
df = r.groupby(by=['label_name', 'model_name', 'input_text'])[['mean_macro_precision',
                                                               'mean_macro_recall', 'mean_macro_f1-score']].mean()
# df.style.format(precision=2).background_gradient(cmap='Blues')
# imodelsx.process_results.delete_runs_in_dataframe(r[r.model_name == 'aug-linear'], actually_delete=True)

In [ ]:
# rotate xticklabels
d = df.reset_index()
d['legend'] = d['input_text'] + ' ' + d['model_name']
ax = sns.barplot(data=d, x='label_name',
                 y='mean_macro_f1-score', hue='legend', palette='Blues', hue_order=sorted(d['legend'].unique()))
plt.xticks(rotation=90)
plt.ylim(bottom=0.5)
# legend to right
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))